In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import csv 
import re
from py2neo import *
import random
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from datetime import datetime
csv.field_size_limit(500 * 1024 * 1024)

131072

In [2]:
#导入文件
pd.set_option('display.max_rows', None)
code = pd.read_csv('instrument_info.csv', dtype=(object))
news = pd.read_csv('ninfotag_202207281734.csv')
news = news.content                                          #导入新闻
hangye = pd.read_csv('股票行业分类20220728.csv',encoding = 'gbk')         
hangye=hangye.replace('.SZ','',regex=True)
hangye=hangye.replace('.BJ','',regex=True)
hangye=hangye.replace('.SH','',regex=True)
name = code.s_info_name
indus_node_name = pd.read_excel('申万行业分类.xlsx')         #导入行业

code.set_index(['s_info_code'])
hangye.set_index(['s_info_code'])
code_merged = pd.merge(code,hangye,on=['s_info_code'])
code_merged = code_merged.drop_duplicates(subset=['s_info_code'])
code_merged=code_merged.reset_index(drop=True) 
code = code_merged.s_info_code
name = code_merged.s_info_name
indus = code_merged.s_indus
indus_node_code_3_data = indus_node_name.s_indus_code
indus_node_code_3_data = list(map(str,indus_node_code_3_data))
indus_node_name_data = indus_node_name.SW三级行业

In [4]:
##连接neo4j数据库，输入地址、用户名、密码
graph = Graph("http://localhost:7474", auth=("neo4j", "123456"))
#graph.delete_all() #清空用

In [ ]:
# 为每个股票创建结点
for i in range(len(name)):
    node = Node ('stock', stock_name=name[i], stock_code=code[i],stock_indus=indus[i])
    graph.create(node)

In [ ]:
# 为每个股票行业板块创建结点
for i in range(len(indus_node_name)):
    node = Node ('三级行业', indus_name=indus_node_name_data[i], indus_code_3=indus_node_code_3_data[i])
    graph.create(node)

In [6]:
#创建所属三级行业关系
for i in range(len(name)):
    matcher = NodeMatcher(graph)
    relationship_matcher = RelationshipMatcher(graph)
    a = matcher.match('stock').where(stock_code=code[i]).first()
    b = matcher.match('三级行业').where(indus_name=a['stock_indus']).first()
    relation = Relationship(a,'属于行业',b)
    graph.create(relation)

In [7]:
#创建新闻共现关系
pattern = re.compile(r'\d+')
file_name = '20220704_20220710.csv'
relation_property = {}
with open(file_name, 'r', encoding='UTF-8') as f: 
    rows = csv.reader(f)
    for row in rows:
        num = pattern.findall(row[3])
        ret = [i for i in set(num) if i in set(code)]
        if len(ret)>1 and len(ret)<10:
            print(ret)
            for j in range(len(ret)):
                matcher = NodeMatcher(graph)
                relationship_matcher = RelationshipMatcher(graph)
                a = matcher.match('stock').where(stock_code=ret[j]).first()
                
                for k in range(1,len(ret)-j):
                    b = matcher.match('stock',stock_code=ret[j+k]).first()
                    if relationship_matcher.match((a,b), r_type=None).first() == None:
                        rela = Relationship(a,'同时在新闻中出现1次',b)
                        relb = Relationship(b,'同时在新闻中出现1次',a)
                        graph.create(rela)
                        graph.create(relb)
                    else:
                        
                        res1 = relationship_matcher.match((a,b), r_type=None).first()
                        res2 = relationship_matcher.match((b,a), r_type=None).first()
                        time = str(res1)
                        pattern_time = "同时在新闻中出现\d+"
                        time_str = re.findall(pattern_time, time)
                        time_str = str(time_str)
                        time_number = re.sub("[^0-9]", "", time_str)
                        time_number = int(time_number)
                        time_number = time_number+1
                        graph.C
                        graph.separate(res2)
                        rela = Relationship(a,'同时在新闻中出现%s次'%time_number,b)
                        relb = Relationship(b,'同时在新闻中出现%s次'%time_number,a)
                        graph.create(rela)
                        graph.create(relb)

['600884', '300035', '688116', '603659']
['300406', '002736']
['603466', '603477']
['300387', '601901']
['603486', '603489']
['603500', '603505', '603508']
['603466', '603477']
['603518', '603515', '603477']
['002747', '300124', '000425', '688017', '000157', '600031']
['002939', '600558', '000061', '300033']
['000333', '002677', '300911', '000651', '300894', '600690']
['002466', '600276', '600519', '000333']
['000807', '002915']
['002800', '000629', '002077', '301201', '300472', '600276']
['300347', '600570', '688599', '300014', '000568']
['600188', '000977', '601966', '600600', '601058', '002078', '600690']
['301158', '688234']
['002183', '601288']
['688063', '300763', '688390', '605117', '688032']
['603567', '603505', '603565', '603508', '603566']
['603569', '603567', '603505', '603565', '603568', '603508', '603566']
['600918', '600109', '600369']
['000559', '000766']
['000061', '000001']
['603518', '603577', '603578']
['603569', '603567', '603565', '603568', '603508', '603566']
['30

['300033', '600710']
['002813', '002326', '600109', '601696']
['600603', '300033']
['300033', '603686']
['300033', '600710']
['300033', '600710']
['300033', '603686']
['300024', '688598', '000629', '002031', '002594', '688033', '002598', '000584', '002978']
['300033', '603686']
['300033', '600710']
['300033', '603686']
['688689', '300033']
['601336', '601319', '601601', '601318', '601628']
['000030', '000061']
['300750', '600519', '600563']
['688689', '300033', '002594']
['688689', '300033']
['688689', '300033']
['688689', '300033']
['300024', '300748']
['601012', '600905', '688303', '600089', '600438', '000591', '601636']
['300033', '605068']
['603259', '600519']
['300033', '605068']
['688689', '300033']
['300033', '600566']
['300033', '600352']
['300033', '603366']
['300033', '688686']
['600308', '300033']
['300033', '600955']
['300033', '600612']
['300033', '688106']
['600308', '300033']
['300033', '688106']
['300033', '601698']
['600308', '300033']
['601577', '300033']
['600720', '

['600395', '300033']
['002176', '002460']
['002594', '002192']
['600848', '605319']
['000409', '300033']
['002673', '600975', '300967', '002234', '600109', '002100', '300498', '002696', '002714']
['603399', '601677', '603876', '600109', '605208', '002192', '002501']
['002201', '600801', '000546', '603737', '301010', '002088']
['002673', '600975', '300967', '002234', '600109', '002100', '300498', '002696', '002714']
['002594', '600727', '002460']
['000762', '002466', '002805', '600338', '002192']
['600276', '688180']
['601857', '603588', '000876', '000783', '002271', '603822', '688087', '688156', '688196']
['300033', '000906']
['600269', '000037', '300033', '000899']
['688075', '603115', '300589', '688311']
['688075', '603115', '300589', '688311']
['300347', '603127']
['000061', '601939', '601658']
['300033', '002953']
['600630', '002083', '600448']
['002643', '300033']
['688220', '600941']
['300750', '002454', '300660']
['000519', '600259', '002240', '002497']
['300033', '002254']
['30

['300760', '002013', '600276', '600765']
['600390', '600104', '600617']
['300024', '600503', '002703', '002909', '601975']
['000858', '000568']
['002176', '002719', '002460']
['600771', '603079', '300677', '600196', '002626', '600380']
['601012', '600481', '601995', '002129', '600109', '600999', '600089', '600438']
['000807', '600383', '000651', '000960', '600048', '600690']
['688071', '688676', '688189', '688037']
['688202', '603127', '002219', '002821', '300347', '300759', '688315']
['000545', '000629', '600727', '300891', '002145', '002136', '002601', '002978']
['300033', '002588']
['300033', '600354']
['300867', '300033']
['300867', '300033']
['300033', '002256']
['002373', '300033']
['002373', '300033']
['300033', '300249']
['300033', '002256']
['300033', '300249']
['600884', '600295']
['000017', '000697']
['300033', '000735']
['300033', '300249']
['002219', '301060', '688315', '002821', '300347', '688202', '603127']
['300033', '601969']
['002063', '688228', '600718']
['300033', '

['600029', '600210', '002594', '600115', '601111']
['688280', '300033']
['000878', '300033']
['603955', '300033']
['300033', '600733']
['600845', '300033']
['300485', '300033']
['300033', '002309']
['600787', '300033']
['300870', '300033']
['603093', '601066']
['603093', '300033']
['300485', '300033']
['002219', '300033']
['300033', '600790']
['300033', '600790']
['300485', '300033']
['000629', '002145', '300059', '002031', '600517', '600438', '000899']
['300015', '300750', '300122', '300274', '300059', '300014']
['300033', '002317']
['300485', '300033']
['600694', '600859', '601888', '000501', '002251', '000759', '601995']
['300033', '002309']
['603259', '688202', '002100', '002714']
['002812', '300033', '600210']
['300485', '300033', '300122']
['300033', '600790']
['603363', '002100', '300498', '603477', '002714']
['688561', '002268', '688023']
['002219', '300033']
['300485', '300033']
['600383', '001914', '002314', '001979', '601377', '600048', '600325', '000002']
['300033', '600733

...

['300033', '300907']
['300033', '002594']
['300033', '002334']
['300033', '000795']
['300033', '002451']
['002594', '600028']
['601555', '601375', '000750', '600030', '601688']
['300033', '002399']
['000564', '300033']
['300033', '002334']
['300033', '002334']
['002594', '300059', '000625', '600893', '601899', '002714']
['601878', '600369', '601995']
['600879', '600081', '002011']
['002695', '603517']
['300750', '002460', '002812', '603799', '002594', '300014']
['603758', '300033']
['601169', '601229', '601009', '600919', '002142', '002807', '601577']
['688202', '688131', '300683']
['600438', '000591']
['300223', '600118', '300207', '688368', '300438', '300624', '603195']
['002460', '002091', '000009', '600456', '300080']
['300379', '688367']
['002530', '002766']
['300750', '002460', '002340', '002009', '603588', '002741', '002034']
['000957', '002031', '002500']
['601066', '600030', '601995']
['000802', '300251', '600977']
['300750', '002466', '002475', '300450']
['300750', '002475', 

['601888', '002163', '600859', '601021', '600628', '601118']
['003031', '300620', '300747', '300548', '300394', '300570']
['600745', '688079', '002456', '688260', '002845']
['002955', '300007']
['300033', '301119']
['300030', '300289', '600196']
['300033', '002428']
['300033', '002381']
['300033', '002334']
['600798', '600575', '601006', '600026', '600495', '002691']
['002580', '300033']
['000027', '601169']
['300033', '000408']
['002456', '002273', '002241', '002962', '002845']
['300033', '300533']
['000050', '000090']
['300033', '002232']
['300033', '000416']
['300303', '300033']
['300127', '000970', '002056', '688122', '002222', '600980']
['300033', '000981']
['300033', '002965']
['000823', '600699', '002920', '000062', '000338', '000636']
['300455', '688066', '300053']
['000925', '002185', '603501', '002079']
['300033', '002038']
['300033', '000981']
['300033', '002038']
['300303', '300033']
['603109', '002139']
['601208', '002498', '300265', '002276', '300481']
['300033', '002920'

['000629', '002145', '000709', '601727']
['301039', '301080', '002402', '688388', '300401', '300616', '300450', '300791']
['601633', '300496', '600104', '600617', '600628']
['300967', '002746', '002234', '300761', '002299']
['002475', '300141', '603290']
['002366', '002248', '600114', '300971']
['601988', '600941']
['603700', '300259', '300066', '300349']
['300724', '300842', '300316', '002129', '600438', '300118', '601865']
['601728', '002209', '600050', '000651']
['000603', '000612', '600392', '000426', '601899']
['000957', '600609', '600519', '600653']
['300750', '600276', '600893', '300601']
['300973', '300033']
['600185', '300033']
['603881', '300033']
['603916', '300033']
['300033', '300219']
['300009', '300033']
['601567', '300033']
['600170', '300033']
['000778', '300033']
['601058', '300033']
['601828', '300033']
['300942', '300033']
['002444', '300033']
['601169', '300033']
['300026', '300033']
['300033', '002572']
['300033', '603871']
['300671', '300033']
['688055', '300033'

['600179', '601919', '603565']
['600837', '000001']
['000722', '600056', '300256']
['000016', '600085', '600436', '000538', '000001']
['000756', '688180', '600062']
['300750', '002939', '688162', '002594', '002074', '688567', '300024', '603895', '002249']
['601066', '000001']
['002466', '000792', '002594', '002747', '300124', '002056', '300832', '002460', '688556']
['000016', '601066']
['601689', '688005', '601128', '600803']
['300383', '002197']
['603799', '600742', '002805']
['600016', '000061']
['000768', '002171']
['002686', '002457', '601669', '002671']
['002173', '601066', '600056']
['600016', '000061']
['603708', '300767', '002621', '300587']
['600645', '002030', '300289', '300406']
['600354', '600598', '000713', '002215']
['000333', '600690']
['601011', '600758', '000690', '000573', '600017', '600397']
['000603', '000751', '000960', '600711', '000758', '600497']
['600763', '603043', '000895', '000683']
['600732', '002218', '002062']
['688187', '605111', '002079', '600104', '688

['000333', '000921', '000651', '600690']
['300750', '002594', '601633', '601127', '000625']
['600519', '600305']
['603043', '603711', '002186', '000721']
['601995', '601066', '601555']
['002673', '601009']
['601766', '600050', '301047', '002594', '000100', '002714']
['300132', '600223', '300957', '300856', '300955']
['601099', '600999', '002500', '002945', '601456']
['002695', '603517']
['300033', '603088']
['600905', '600163', '601016', '603693', '000591', '601619']
['301046', '300490']
['002053', '002216']
['600789', '002603', '300358']
['002242', '000858', '603833']
['002240', '002124', '002422', '000061', '300390', '601225']
['300059', '002736', '002966', '601788']
['300390', '601127', '000600', '603015', '688676', '600418', '000755', '002031']
['603138', '300832']
['002039', '000050', '300260']


In [8]:
#更新过去一周新闻出现总数
node_matcher = NodeMatcher(graph)                                      
relationship_matcher = RelationshipMatcher(graph)
for n in range(len(name)):
    a = node_matcher.match('stock', stock_code = code_merged.s_info_code[n]).first()
    relationship = list(relationship_matcher.match([a], r_type=None))
    time = str(relationship)
    time_str = re.findall(pattern_time, time)
    time_number_sum = 0
    if len(time_str):
        for l in range(len(time_str)):
            time_number = re.sub("[^0-9]","",time_str[l])
            time_number = int(time_number)
            time_number_sum = time_number_sum + time_number
        a.update({'过去一周新闻出现总次数': time_number_sum})
        a.update({'过去一周新闻新闻共现公司总数': len(time_str)})
        graph.push(a)

In [ ]:
for i in range(len(name)):
    a = node_matcher.match('stock', stock_code = code[i]).first()
    print(i)
    relationship = list(relationship_matcher.match([a], r_type='同时在新闻中出现1次'))
    for j in range(len(relationship)):
        rel = relationship[j]
        graph.separate(rel)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27